In [1]:
from nltk.tokenize import sent_tokenize
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import glob
import os
from os import path
import string
import numpy
import re
from __future__ import absolute_import, division, print_function
import pandas as pd
import multiprocessing
import gensim
import random
from tqdm import tqdm
from sklearn.manifold import TSNE
from gensim.models import KeyedVectors
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
os.chdir("D://Scriptie//Data//lines//cleaned")
list_sample_txt = glob.glob('*.txt')
list_sample_txt = [f for f in list_sample_txt if f[0:4] == 'peri']
list_sample_txt = list_sample_txt[5:10]
list_sample_txt

['period_sample_1865.txt',
 'period_sample_1875.txt',
 'period_sample_1885.txt',
 'period_sample_1895.txt',
 'period_sample_1905.txt']

In [4]:
def Txt_Lines(txt):
    with open(txt, encoding = 'utf-8') as f:
            content = f.read().split('\n')
    
    lines = list()
    
    for line in content:
        tmp = line.split(' ')
        lines.append(tmp)
    
    return lines

In [5]:
for txt in list_sample_txt:
    sentences = Txt_Lines(txt)
    model = Word2Vec(sentences, size=150, window=12, iter = 5, min_count=100, workers=6)
    model.train(sentences,total_examples=len(sentences), epochs=10)
    model.wv.save_word2vec_format(txt[14:18] +"_static_model.bin", binary=True)
    print(txt + ' modelled')

period_sample_1865.txt modelled
period_sample_1875.txt modelled
period_sample_1885.txt modelled
period_sample_1895.txt modelled
period_sample_1905.txt modelled


In [7]:
model.most_similar('buitenland',topn=20)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('duitschland', 0.5407097935676575),
 ('nederland', 0.5337533354759216),
 ('duitsohland', 0.5188031792640686),
 ('belgië', 0.5126810669898987),
 ('engeland', 0.5000225305557251),
 ('buitenlandsche', 0.49754172563552856),
 ('duitechland', 0.4850339889526367),
 ('duitsehland', 0.4811227023601532),
 ('duitschiand', 0.4749215543270111),
 ('amerika', 0.4608646035194397),
 ('binnenland', 0.45768994092941284),
 ('frankrijk', 0.4542026221752167),
 ('buiteni', 0.4495086073875427),
 ('land', 0.43977653980255127),
 ('zwitserland', 0.43865296244621277),
 ('duiteche', 0.43229857087135315),
 ('handelsverkeer', 0.4252898395061493),
 ('canada', 0.4195053279399872),
 ('rusland', 0.41424164175987244),
 ('europa', 0.4036165475845337)]

In [2]:
os.chdir("D://Scriptie//Data//lines//cleaned")
list_mod = glob.glob('*_static_model.bin')
len(list_mod)

18

In [3]:
def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
	# patch by Richard So [https://twitter.com/richardjeanso) (thanks!) to update this code for new version of gensim
	base_embed.init_sims()
	other_embed.init_sims()

	# make sure vocabulary and indices are aligned
	in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)

	# get the embedding matrices
	base_vecs = in_base_embed.syn0norm
	other_vecs = in_other_embed.syn0norm

	# just a matrix dot product with numpy
	m = other_vecs.T.dot(base_vecs) 
	# SVD method from numpy
	u, _, v = np.linalg.svd(m)
	# another matrix operation
	ortho = u.dot(v) 
	# Replace original array with modified one
	# i.e. multiplying the embedding matrix (syn0norm)by "ortho"
	other_embed.syn0norm = other_embed.syn0 = (other_embed.syn0norm).dot(ortho)
	return other_embed

In [4]:
def intersection_align_gensim(m1,m2, words=None):
	# Get the vocab for each model
	vocab_m1 = set(m1.vocab.keys())
	vocab_m2 = set(m2.vocab.keys())

	# Find the common vocabulary
	common_vocab = vocab_m1&vocab_m2
	if words: common_vocab&=set(words)

	# If no alignment necessary because vocab is identical...
	if not vocab_m1-common_vocab and not vocab_m2-common_vocab:
		return (m1,m2)

	# Otherwise sort by frequency (summed for both)
	common_vocab = list(common_vocab)
	common_vocab.sort(key=lambda w: m1.vocab[w].count + m2.vocab[w].count,reverse=True)

	# Then for each model...
	for m in [m1,m2]:
		# Replace old syn0norm array with new one (with common vocab)
		indices = [m.vocab[w].index for w in common_vocab]
		old_arr = m.syn0norm
		new_arr = np.array([old_arr[index] for index in indices])
		m.syn0norm = m.syn0 = new_arr

		# Replace old vocab dictionary with new one (with common vocab)
		# and old index2word with new one
		m.index2word = common_vocab
		old_vocab = m.vocab
		new_vocab = {}
		for new_index,word in enumerate(common_vocab):
			old_vocab_obj=old_vocab[word]
			new_vocab[word] = gensim.models.word2vec.Vocab(index=new_index, count=old_vocab_obj.count)
		m.vocab = new_vocab

	return (m1,m2)

In [6]:
for num,txt in enumerate(list_mod):
    if num == 0:
        mod_prev = KeyedVectors.load_word2vec_format(txt, binary = True)
        print('model ' + str(num) + ' loaded')
    if num > 0:
        mod_next = KeyedVectors.load_word2vec_format(txt, binary = True)
        smart_procrustes_align_gensim(mod_prev, mod_next, words=None)
        
        mod_prev.wv.save_word2vec_format(list_mod[num-1][0:4] + '_static_aligned.bin', binary=True)
        mod_next.wv.save_word2vec_format(txt[0:4] + '_static_aligned.bin', binary=True)
        
        mod_prev = mod_next
        print('aligned, on to the next')

model 0 loaded


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).


aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
aligned, on to the next
